%%HTML
<style>

.CodeMirror {
    width: 100vw;
}

.container {
    width: 99% !important;
}

.rendered_html {
  font-size:0.7em;
}
.rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
     font-size: 100%;
}

</style>

## Flashing device

http://micropython.org/download

`pip install esptool`

## ESP32

- Internal 4 MB RAM?

ESP32 can use external SPIRAM (psRAM) to expand available RAM up to 16MB [source](https://github.com/loboris/MicroPython_ESP32_psRAM_LoBo).

In [1]:
!esptool.py --port /dev/ttyUSB0 --chip esp32 erase_flash

esptool.py v2.1
Connecting......
Chip is ESP32D0WDQ6 (revision 1)
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 3.6s
Hard resetting...


In [2]:
!esptool.py --chip esp32 --port /dev/micropython write_flash -z 0x1000 esp32-20180613-v1.9.4-156-g03590641.bin

esptool.py v2.1
Connecting.......
Chip is ESP32D0WDQ6 (revision 1)
Uploading stub...
Running stub...
Stub running...
Configuring flash size...
Auto-detected Flash size: 4MB
Compressed 1016496 bytes to 634306...
Wrote 1016496 bytes (634306 compressed) at 0x00001000 in 55.9 seconds (effective 145.5 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting...


# Communication

## Serial port communication

Universal asynchronous receiver-transmitter (UART)

- serial - parallel (1 bit <-> bytes)
- baudrate (bits per second) (default 115200)

- Tools
  - screen
  - Putty
  - Picocom
  - YAT (Yet Another Terminal)
  - pySerial
  - Jupyter Notebook
  

## Bind tty to static name (Linux)

/etc/udev/rules.d/10-local.rules

`ACTION=="add", ATTRS{idVendor}=="0403", ATTRS{idProduct}=="6001", SYMLINK+="my_uart"`

You can check for the variables of your device by running

`udevadm info --name=/dev/ttyUSBx --attribute-walk`

 ATTRS{idProduct}=="0002"
 ATTRS{idVendor}=="1d6b"

## File transfer

### ampy

`pip install adafruit-ampy`

`ampy -p /dev/ttyUSB0 put boot.py`


In [4]:
!ampy -p /dev/micropython ls

boot.py


## WebREPL


![webrepl](https://cdn-learn.adafruit.com/assets/assets/000/035/130/original/microcontrollers_Screen_Shot_2016-08-25_at_6.41.48_PM.png?1472175855)

`import webrepl_setup`


http://docs.micropython.org/en/latest/esp8266/esp8266/quickref.html

`import webrepl_setup`

- uses a single connection over WebSocket

If the board isn't connected already to a wireless access point then it will create one which you can connect to from your computer.

## REPL (read-eval-print-loop)

- help(), help('modules'), dir()
- auto-indent, auto-completion
- Interrupting
- paste mode
- bitbucket variable

In [3]:
help('modules')

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name '_' is not defined


In [4]:
import esp
print(esp.flash_size())

4194304


## Reset
***
__soft__ `CTRL+D`
- clears the state of the MicroPython virtual machine
- leaves hardware peripherals unaffected
  
```
import sys
sys.exit()
```
***
**hard**

- same as performing a power cycle to the boar

```
import machine
machine.reset()
```

#### Startup code

- boot.py: power cycle, hard reset, deep sleep
- main.py: your main program

In [5]:
with open('boot.py', 'r') as f:
    print(f.read())

# This file is executed on every boot (including wake-boot from deepsleep)
#import esp
#esp.osdebug(None)
#import webrepl
#webrepl.start()



## Machine module

In [6]:
import machine
print(help(machine))

object <module 'umachine'> is of type module
  __name__ -- umachine
  mem8 -- <8-bit memory>
  mem16 -- <16-bit memory>
  mem32 -- <32-bit memory>
  freq -- <function>
  reset -- <function>
  unique_id -- <function>
  sleep -- <function>
  deepsleep -- <function>
  idle -- <function>
  disable_irq -- <function>
  enable_irq -- <function>
  time_pulse_us -- <function>
  Timer -- <class 'Timer'>
  WDT -- <class 'WDT'>
  SLEEP -- 2
  DEEPSLEEP -- 4
  Pin -- <class 'Pin'>
  Signal -- <class 'Signal'>
  TouchPad -- <class 'TouchPad'>
  ADC -- <class 'ADC'>
  DAC -- <class 'DAC'>
  I2C -- <class 'I2C'>
  PWM -- <class 'PWM'>
  RTC -- <class 'RTC'>
  SPI -- <class 'SoftSPI'>
  UART -- <class 'UART'>
  reset_cause -- <function>
  HARD_RESET -- 2
  PWRON_RESET -- 1
  WDT_RESET -- 3
  DEEPSLEEP_RESET -- 4
  SOFT_RESET -- 5
  wake_reason -- <function>
  PIN_WAKE -- 1
  EXT0_WAKE -- 1
  EXT1_WAKE -- 2
  TIMER_WAKE -- 3
  TOUCHPAD_WAKE -- 4
  ULP_WAKE -- 5
None


### RTC (real time clock)

In [7]:
rtc = machine.RTC()
rtc.init((2014, 5, 1, 4, 13, 0, 0, 0))
print(rtc.datetime())

(2014, 5, 1, 3, 13, 0, 0, 155)


### Power control

In [ ]:
import machine

rtc = machine.RTC()
rtc.irq(trigger=rtc.ALARM0, wake=machine.DEEPSLEEP)
rtc.alarm(rtc.ALARM0, 10000)
machine.deepsleep()
if machine.reset_cause() == machine.DEEPSLEEP_RESET:
    print("WOKE")
else:
    machine.reset()

#### WiFi

In [ ]:
# Access Point
import network
ap_if = network.WLAN(network.AP_IF)
ap_if.active()

In [ ]:
# Connect to network
import network

sta_if = network.WLAN(network.STA_IF)
print('connecting to network...')
sta_if.active(True)
sta_if.connect('NDC 2018', '')
while not sta_if.isconnected():
    pass
print('network config:', sta_if.ifconfig())

### upip (package manager)

In [ ]:
import upip
upip.install('micropython-collections')